# DRINKING WATER FOR ALL - Pré-traitement des données - Etude sur l'accès à l'eau potable dans le monde




# Introduction


In [1]:
## Importation des modules & librairies

import numpy as np # Calculs numériques et opérations mathématiques
import pandas as pd # Manipulation et analyse de données

## Paramètres d'affichages
pd.set_option('display.float_format', lambda x: '%.0f' % x) # Affichage numérique

In [2]:
## Mise en place des fonctions
def explore_dataframe(data_to_analyse):
    """
    Effectue une analyse exploratoire basique sur un dataframe donné.

    Parameters:
    data_to_analyse (DataFrame): Le dataframe à analyser.
    """
    # Dimensions du dataframe
    print('*' * 40 + " DIMENSIONS " + '*' * 44)
    print(f"Le dataframe comporte {data_to_analyse.shape[1]} variable(s) pour {data_to_analyse.shape[0]} observation(s).")

    # Typologies des variables
    print('*' * 40 + " TYPOLOGIES " + '*' * 44)
    for col, dtype in data_to_analyse.dtypes.items():
        print(f"{col} : {dtype}")

    # Effectifs des variables
    print('*' * 40 + " EFFECTIFS " + '*' * 45)
    for col, count in data_to_analyse.describe(include='all').loc['count'].items():
        print(f"{col} : {count}")

    # Valeurs nulles dans le dataframe
    print('*' * 40 + " VALEURS NULLES " + '*' * 40)
    for col, count in data_to_analyse.isnull().sum().items():
        print(f"{col} : {count}")

    # Duplicats dans le dataframe
    print('*' * 40 + " DUPLICATS " + '*' * 45)
    for col, count in data_to_analyse[data_to_analyse.duplicated()].count().items():
        print(f"{col} : {count}")


In [3]:
## Importation des données initiales
region_data = pd.read_csv("RegionCountry.csv", sep=',')
population_data = pd.read_csv("Population.csv", sep=',')
political_data = pd.read_csv("PoliticalStability.csv", sep=',')
mortality_data = pd.read_csv("MortalityRateAttributedToWater.csv", sep=',')
safety_data = pd.read_csv("BasicAndSafelyManagedDrinkingWaterServices.csv", sep=',')

# Pré-traitement des données

## Données géographiques

In [4]:
## Analyse exploratoire
explore_dataframe(region_data)

**************************************** DIMENSIONS ********************************************
Le dataframe comporte 2 variable(s) pour 194 observation(s).
**************************************** TYPOLOGIES ********************************************
REGION (DISPLAY) : object
COUNTRY (DISPLAY) : object
**************************************** EFFECTIFS *********************************************
REGION (DISPLAY) : 194
COUNTRY (DISPLAY) : 194
**************************************** VALEURS NULLES ****************************************
REGION (DISPLAY) : 0
COUNTRY (DISPLAY) : 0
**************************************** DUPLICATS *********************************************
REGION (DISPLAY) : 0
COUNTRY (DISPLAY) : 0


In [5]:
## Préparation des données

# Modifier la typologie des variables
region_data['REGION (DISPLAY)'] = region_data['REGION (DISPLAY)'].astype('string')
region_data['COUNTRY (DISPLAY)'] = region_data['COUNTRY (DISPLAY)'].astype('string')

# Modifier le nom des variables
region_data.rename(columns={'REGION (DISPLAY)': 'Region', 'COUNTRY (DISPLAY)': 'Pays'}, inplace=True)

In [6]:
## Liste des régions

# Identification des régions référencées
liste_regions = region_data["Region"].unique()
regions_df = pd.DataFrame(liste_regions, columns=["Region"])

# Aperçu des résultats
regions_df

,Region
0,Europe
1,Western Pacific
2,Eastern Mediterranean
3,South-East Asia
4,Africa
5,Americas


In [7]:
## Liste des pays
# region_data['Pays']

# Enregistrement de la liste des pays référencés
liste_pays_region = region_data['Pays']

# Aperçu du nombre de pays référencés
print("Nombre de pays référencés :", len(liste_pays_region))

Nombre de pays référencés : 194


## Données géo-politiques

In [8]:
## Analyse exploratoire
explore_dataframe(political_data)

**************************************** DIMENSIONS ********************************************
Le dataframe comporte 4 variable(s) pour 3526 observation(s).
**************************************** TYPOLOGIES ********************************************
Country : object
Year : int64
Political_Stability : float64
Granularity : object
**************************************** EFFECTIFS *********************************************
Country : 3526
Year : 3526.0
Political_Stability : 3526.0
Granularity : 3526
**************************************** VALEURS NULLES ****************************************
Country : 0
Year : 0
Political_Stability : 0
Granularity : 0
**************************************** DUPLICATS *********************************************
Country : 0
Year : 0
Political_Stability : 0
Granularity : 0


In [9]:
## Préparation des données

# Vérification du nombre de valeurs uniques au sein de la variable 'Granularity'
# political_data["Granularity"].unique()
# Suppression de la variable non pertinente
political_data = political_data.drop("Granularity", axis=1)

# Modifier la typologie des variables
political_data['Country'] = political_data['Country'].astype('string')

# Modifier le nom des variables
political_data.rename(columns={'Country': 'Pays', 'Year': 'Année', 'Political_Stability': 'Score_Stabilité_Politique'}, inplace=True)

# Formatage des noms de pays
political_data.loc[(political_data['Pays'] == 'China, mainland'), 'Pays'] = 'China'

In [10]:
## Liste des pays

# Identification des pays non référencés par le fichier contenant les données géographiques
pays_non_listes = political_data[~political_data['Pays'].isin(liste_pays_region)]['Pays'].unique()

# Calcul du nombre de pays non référencés
nombre_pays_non_listes = len(pays_non_listes)

# Aperçu de la liste des pays non référencés
print("Nombre de pays non référencés:", nombre_pays_non_listes)
print("")
print("Liste des pays non référencés:")
print("")
for pays in pays_non_listes:
    print(pays)
print("")

# Suppression des pays non référencés
political_data = political_data[~political_data['Pays'].isin(pays_non_listes)]
political_data.reset_index(drop=True, inplace=True)

# Calcul du nombre de pays référencés
nbr_pays = political_data['Pays'].nunique()

# Aperçu du résultat
print("Nombre de pays référencés :", nbr_pays)
print("")

# Aperçu de la valeur minimale de Score_Stabilité_Politique
min_score = political_data['Score_Stabilité_Politique'].min()
print("Valeur minimale de Score_Stabilité_Politique :", min_score)
# Aperçu de la valeur maximale de Score_Stabilité_Politique
max_score = political_data['Score_Stabilité_Politique'].max()
print("Valeur maximale de Score_Stabilité_Politique :", max_score)

Nombre de pays non référencés: 9

Liste des pays non référencés:

American Samoa
Bermuda
China, Hong Kong SAR
China, Macao SAR
China, Taiwan Province of
Greenland
North Macedonia
Palestine
Puerto Rico

Nombre de pays référencés : 191

Valeur minimale de Score_Stabilité_Politique : -3.31
Valeur maximale de Score_Stabilité_Politique : 1.76


## Données démographiques

In [11]:
## Analyse exploratoire
explore_dataframe(population_data)

**************************************** DIMENSIONS ********************************************
Le dataframe comporte 4 variable(s) pour 20914 observation(s).
**************************************** TYPOLOGIES ********************************************
Country : object
Granularity : object
Year : int64
Population : float64
**************************************** EFFECTIFS *********************************************
Country : 20914
Granularity : 20914
Year : 20914.0
Population : 20914.0
**************************************** VALEURS NULLES ****************************************
Country : 0
Granularity : 0
Year : 0
Population : 0
**************************************** DUPLICATS *********************************************
Country : 0
Granularity : 0
Year : 0
Population : 0


In [12]:
## Préparation des données

# Faire pivoter le dataframe
population_data = population_data.pivot(index=['Country','Year'], columns='Granularity', values='Population').reset_index()

# Indexation du nouveau dataframe
population_data = population_data.rename_axis(columns={'Granularity': 'index'})
population_data.columns.name = None # Réinitialiser les noms des variables

# Formatage des données
population_data['Female'] = population_data['Female'] * 1000
population_data['Male'] = population_data['Male'] * 1000
population_data['Rural'] = population_data['Rural'] * 1000
population_data['Urban'] = population_data['Urban'] * 1000
population_data['Total'] = population_data['Total'] * 1000

# Ordre des variables
population_data = population_data[['Country', 'Year', 'Female','Male','Rural','Urban','Total']].copy()

# Modifier le nom des variables
population_data.rename(columns={'Country': 'Pays', 'Year': 'Année','Female': 'Female_Population','Male': 'Male_Population','Rural': 'Rural_Population','Urban': 'Urban_Population','Total': 'Total_Population',}, inplace=True)

In [13]:
## Liste des pays

# Identification des pays non référencés par le fichier contenant les données géographiques
pays_non_listes = population_data[~population_data['Pays'].isin(liste_pays_region)]['Pays'].unique()

# Calcul du nombre de pays non référencés
nombre_pays_non_listes = len(pays_non_listes)

# Aperçu de la liste des pays non référencés
print("Nombre de pays non référencés:", nombre_pays_non_listes)
print("")
print("Liste des pays non référencés:")
print("")
for pays in pays_non_listes:
    print(pays)
print("")

# Suppression des pays non référencés
population_data = population_data[~population_data['Pays'].isin(pays_non_listes)]
population_data.reset_index(drop=True, inplace=True)

# Calcul du nombre de pays référencés
nbr_pays = population_data['Pays'].nunique()

# Aperçu du résultat
print("Nombre de pays référencés :", nbr_pays)
print("")

Nombre de pays non référencés: 46

Liste des pays non référencés:

American Samoa
Anguilla
Aruba
Bermuda
Bonaire, Sint Eustatius and Saba
British Virgin Islands
Cayman Islands
Channel Islands
China, Hong Kong SAR
China, Macao SAR
China, Taiwan Province of
China, mainland
Curaçao
Falkland Islands (Malvinas)
Faroe Islands
French Guyana
French Polynesia
Gibraltar
Greenland
Guadeloupe
Guam
Holy See
Isle of Man
Liechtenstein
Martinique
Mayotte
Montserrat
Netherlands Antilles (former)
New Caledonia
North Macedonia
Northern Mariana Islands
Palestine
Puerto Rico
Réunion
Saint Barthélemy
Saint Helena, Ascension and Tristan da Cunha
Saint Pierre and Miquelon
Saint-Martin (French part)
Serbia and Montenegro
Sint Maarten  (Dutch part)
Sudan (former)
Tokelau
Turks and Caicos Islands
United States Virgin Islands
Wallis and Futuna Islands
Western Sahara

Nombre de pays référencés : 193



In [14]:
## Gestion des valeurs nulles

# Filtrage des pays concernés par les valeurs nulles au sein des variables Male ou Female
pays_valeurs_nulles = population_data[(population_data['Male_Population'].isnull()) | (population_data['Female_Population'].isnull())]['Pays']

# Recupération de la liste des pays
pays_valeurs_nulles_unique = pays_valeurs_nulles.unique()

# Aperçu des pays concernés par les valeurs nulles
print("Pays concernés par les valeurs nulles au sein des variables Male ou Female :")
for pays in pays_valeurs_nulles_unique:
    print(pays)

# Suppression des lignes correspondantes
indices_a_supprimer = population_data[(population_data['Male_Population'].isnull()) | (population_data['Female_Population'].isnull())].index
population_data = population_data.drop(indices_a_supprimer)

Pays concernés par les valeurs nulles au sein des variables Male ou Female :
Andorra
Cook Islands
Dominica
Marshall Islands
Monaco
Nauru
Niue
Palau
Saint Kitts and Nevis
San Marino
Tuvalu


In [15]:
# Modifier la typologie des variables
population_data['Pays'] = population_data['Pays'].astype('string')
population_data['Female_Population'] = population_data['Female_Population'].astype('int')
population_data['Male_Population'] = population_data['Male_Population'].astype('int')
population_data['Rural_Population'] = population_data['Rural_Population'].astype('int')
population_data['Urban_Population'] = population_data['Urban_Population'].astype('int')
population_data['Total_Population'] = population_data['Total_Population'].astype('int')

# Vérification de la cohérence des données (Total_Population est égal à Male_Population + Female_Population)
mask = population_data['Total_Population'] != (population_data['Male_Population'] + population_data['Female_Population'])

# Mettre à jour Total_Population si nécessaire
population_data.loc[mask, 'Total_Population'] = population_data['Male_Population'] + population_data['Female_Population']

In [16]:
# Calcul de la population totale en 2016
population_data[population_data['Année'] == 2016]['Total_Population'].sum()

# Filtrer les données pour l'année 2016
df_2016 = population_data[population_data['Année'] == 2016]

# Créer un nouveau DataFrame avec les pays uniques et leur population en 2016
df_population_2016 = df_2016[['Pays', 'Total_Population']].drop_duplicates()

## Données relatives à l'accès à l'eau potable

In [17]:
## Analyse exploratoire
explore_dataframe(safety_data)

**************************************** DIMENSIONS ********************************************
Le dataframe comporte 5 variable(s) pour 10476 observation(s).
**************************************** TYPOLOGIES ********************************************
Year : int64
Country : object
Granularity : object
Population using at least basic drinking-water services (%) : float64
Population using safely managed drinking-water services (%) : float64
**************************************** EFFECTIFS *********************************************
Year : 10476.0
Country : 10476
Granularity : 10476
Population using at least basic drinking-water services (%) : 9415.0
Population using safely managed drinking-water services (%) : 3286.0
**************************************** VALEURS NULLES ****************************************
Year : 0
Country : 0
Granularity : 0
Population using at least basic drinking-water services (%) : 1061
Population using safely managed drinking-water services (%) : 719

In [18]:
## Préparation des données

# Modifier la typologie des variables
safety_data['Country'] = safety_data['Country'].astype('string')
safety_data['Granularity'] = safety_data['Granularity'].astype('string')

# Modifier le nom des variables
safety_data.rename(columns={'Country': 'Pays', 'Year': 'Année', 'Granularity': 'Granularité', 'Population using at least basic drinking-water services (%)': '%_Basic_Water','Population using safely managed drinking-water services (%)': '%_Safe_Water',}, inplace=True)

# Faire pivoter le dataframe
safety_data_basic = safety_data.pivot(index=['Pays','Année'], columns='Granularité', values='%_Basic_Water').reset_index()

# Réinitialiser les noms de variables
safety_data_basic = safety_data_basic.rename_axis(columns={'Granularité': 'index'})
safety_data_basic.columns.name = None

# Filtrage des données pour obtenir les chiffres d'accès à l'eau potable
safety_data_safe= safety_data.loc[safety_data['Granularité'] == 'Total'][['Année', 'Pays', '%_Safe_Water']]

# Fusion des deux dataframes
safety_data = pd.merge(safety_data_basic, safety_data_safe, on=['Année','Pays'])

# Modifier le nom des variables
safety_data.rename(columns={'Rural': '%_Basic_Water_Rural', 'Total':'%_Basic_Water_Total', 'Urban': '%_Basic_Water_Urban',
                             "Safety Water":'%_Safe_Water_Total'}, inplace=True)

# Ré-organiser l'ordre des variables pour une meilleur lisibilité
nouvel_ordre_colonnes = ['Pays', 'Année', '%_Basic_Water_Rural', '%_Basic_Water_Urban', '%_Basic_Water_Total', '%_Safe_Water']
safety_data = safety_data.reindex(columns=nouvel_ordre_colonnes)

In [19]:
## Liste des pays

# Identification des pays non référencés par le fichier contenant les données géographiques
pays_non_listes = safety_data[~safety_data['Pays'].isin(liste_pays_region)]['Pays'].unique()

# Calcul du nombre de pays non référencés
nombre_pays_non_listes = len(pays_non_listes)

# Aperçu de la liste des pays non référencés
print("Nombre de pays non référencés:", nombre_pays_non_listes)
print("")

# Calcul du nombre de pays référencés
nbr_pays = safety_data['Pays'].nunique()

# Aperçu du résultat
print("Nombre de pays : référencés", nbr_pays)
print("")

Nombre de pays non référencés: 0

Nombre de pays : référencés 194



## Données relatives aux taux de mortalité dûs à l'eau insalubre

In [20]:
## Analyse exploratoire
explore_dataframe(mortality_data)

**************************************** DIMENSIONS ********************************************
Le dataframe comporte 5 variable(s) pour 549 observation(s).
**************************************** TYPOLOGIES ********************************************
Year : int64
Country : object
Granularity : object
Mortality rate attributed to exposure to unsafe WASH services : float64
WASH deaths : float64
**************************************** EFFECTIFS *********************************************
Year : 549.0
Country : 549
Granularity : 549
Mortality rate attributed to exposure to unsafe WASH services : 549.0
WASH deaths : 183.0
**************************************** VALEURS NULLES ****************************************
Year : 0
Country : 0
Granularity : 0
Mortality rate attributed to exposure to unsafe WASH services : 0
WASH deaths : 366
**************************************** DUPLICATS *********************************************
Year : 0
Country : 0
Granularity : 0
Mortality rate a

In [21]:
## Préparation des données

# Modifier la typologie des variables
mortality_data['Country'] = mortality_data['Country'].astype('string')
mortality_data['Granularity'] = mortality_data['Granularity'].astype('string')

# Faire pivoter le dataframe
mortality_data_1 = mortality_data.pivot(index=['Country','Year'], columns='Granularity', values='Mortality rate attributed to exposure to unsafe WASH services').reset_index()

# Ré-indexaction des variables
mortality_data_1 = mortality_data_1.rename_axis(columns={'Granularity': 'index'})
mortality_data_1.columns.name = None

# Filtrage du nombre de décès
nombre_deces_2016 = mortality_data.loc[mortality_data['Granularity'] == 'Total'][['Year', 'Country', 'WASH deaths']]

# Fusion des deux dataframes
mortality_data = pd.merge(mortality_data_1, nombre_deces_2016, on=['Country','Year'])
mortality_data.rename(columns={'Country': 'Pays','Year': 'Année','Female': 'Female_Death_Per100K', 'Male':'Male_Death_Per100K', 'Total':'Total_Death_Per100K', 'WASH deaths' :'Nbr_Death'}, inplace=True)


In [22]:
## Liste des pays

# Identification des pays non référencés par le fichier contenant les données géographiques
pays_non_listes = mortality_data[~mortality_data['Pays'].isin(liste_pays_region)]['Pays'].unique()

# Calcul du nombre de pays non référencés
nombre_pays_non_listes = len(pays_non_listes)

# Aperçu de la liste des pays non référencés
print("Nombre de pays non référencés:", nombre_pays_non_listes)
print("")

# Calcul du nombre de pays référencés
nbr_pays = mortality_data['Pays'].nunique()

# Aperçu du résultat
print("Nombre de pays référencés :", nbr_pays)
print("")

Nombre de pays non référencés: 0

Nombre de pays référencés : 183



# Exportation des données

In [23]:
# Exportation des fichiers modifiés au format CSV
region_data.to_csv('region_data.csv', index=False)
population_data.to_csv('population_data.csv', index=False)
safety_data.to_csv('safety_data.csv', index=False)
mortality_data.to_csv('mortality_data.csv', index=False)
political_data.to_csv('political_data.csv', index=False)